In [1]:
import pandas as pd
import os

from glob import glob

from sshtunnel import SSHTunnelForwarder

import paramiko
import geopandas as gpd
import numpy as np
import json
import rasterio
import sshtunnel
import MySQLdb

from sqlalchemy import (
    Connection,
    CursorResult,
    Engine,
    Select,
    TextClause,
    create_engine,
    func,
    select,
)

In [2]:
gdf_files = glob(os.path.join('./dataframes', '*', 'gdf*.json'))  # First level
gdf_files.extend(glob(os.path.join('./dataframes', 'gdf*.json')))  # Base directory


In [3]:
gdf_data = pd.DataFrame()

df_list = []

for file in gdf_files:

    temp_data = pd.read_json(file)

    df_list.append(temp_data)

gdf_data = pd.concat(df_list)
gdf_data.reset_index()

,index,time,icao24,lat,lon,velocity,heading,callsign,onground,squawk,...,hour,ref,inf_flt,inf_pt,dist,gnd_elev,gnd_inf_flt,gnd_inf_pt,min_hgt,lidar_min_alt
0,0,1659957628,3dd3bf,50.896539,6.991425,40.222152,339.015102,DHALB,false,7043,...,1659956400,3dd3bf_1_080822,True,False,0.000000,48.438858,True,True,348.438858,NaN
1,1,1659957629,3dd3bf,50.896729,6.991313,40.702893,339.274441,DHALB,false,7043,...,1659956400,3dd3bf_1_080822,True,False,22.452396,42.842587,True,True,342.842587,NaN
2,2,1659957630,3dd3bf,50.897141,6.991016,40.523691,339.954751,DHALB,false,7043,...,1659956400,3dd3bf_1_080822,True,False,72.774262,38.378052,True,True,338.378052,NaN
3,3,1659957631,3dd3bf,50.897470,6.990891,41.007351,340.201124,DHALB,false,7043,...,1659956400,3dd3bf_1_080822,True,False,110.486456,37.173000,True,False,337.173000,NaN
4,4,1659957632,3dd3bf,50.897827,6.990645,41.007351,340.201124,DHALB,false,7043,...,1659956400,3dd3bf_1_080822,True,False,153.742445,37.172001,True,False,337.172001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380479,2150,1698157708,3ff8d0,50.897186,6.921906,40.510628,310.364537,DMFJS,False,7000,...,1698156000,3ff8d0_1_241023,False,False,127.968517,63.442432,False,False,363.442432,395.51
380480,2151,1698157709,3ff8d0,50.897424,6.921463,40.510628,310.364537,DMFJS,False,7000,...,1698156000,3ff8d0_1_241023,False,False,168.756031,63.539776,False,False,363.539776,395.51
380481,2152,1698157710,3ff8d0,50.897598,6.921090,40.903960,309.897835,DMFJS,False,7000,...,1698156000,3ff8d0_1_241023,False,False,201.343082,62.125084,False,False,362.125084,395.51
380482,2153,1698157711,3ff8d0,50.897873,6.920644,41.299954,309.440053,DMFJS,False,7000,...,1698156000,3ff8d0_1_241023,False,False,245.028448,59.539700,False,False,359.539700,395.51


In [4]:
gdf_data['positionnac'] = 10
gdf_data['version'] = 2

In [2]:

MYSQL_secrets_json = './mysql_secrets.json'


with open(MYSQL_secrets_json) as MYSQL_secrets:
  MYSQL_creds = json.load(MYSQL_secrets)


In [6]:
gdf_data = gdf_data.drop(columns=['hour', 'lidar_min_alt'])

In [7]:
ed25519_key = paramiko.Ed25519Key(filename='/home/dimitri/.ssh/id_ed25519')

# Set up the SSH tunnel with the RSA key
with SSHTunnelForwarder(
    (MYSQL_creds['SSH_ADDRESS'], 22),
    ssh_username=MYSQL_creds['SSH_USERNAME'],
    ssh_pkey=ed25519_key,  # Use the loaded RSA key
    remote_bind_address=(MYSQL_creds['REMOTE_BIND_ADDRESS'], MYSQL_creds['REMOTE_BIND_PORT']),
    allow_agent=False,
) as tunnel:
    
    print('connected')

    engstr =  'mysql+pymysql://' + MYSQL_creds['SSH_USERNAME'] + ':' + MYSQL_creds['PYANYWHERE_PASSWORD'] + '@127.0.0.1:' + str(tunnel.local_bind_port)+'/dme3$' + MYSQL_creds['PROD_DATABASE_NAME']

    engine = create_engine(engstr)

    print('step 1')
    
    gdf_data.to_sql(con=engine, name = MYSQL_creds['MAIN_PROD_TABLE_NAME'], if_exists='append')

    print("Done")

connected
step 1
Done


# Infraction tables

In [3]:
inf_files = glob(os.path.join('./dataframes', '*', 'clean_inf_*.json'))  # First level
inf_files.extend(glob(os.path.join('./dataframes', 'clean_inf*.json')))  # Base directory
gnd_inf_files = glob(os.path.join('./dataframes', '*', 'clean_gnd_inf_*.json'))  # First level
gnd_inf_files.extend(glob(os.path.join('./dataframes', 'clean_gnd_inf*.json')))  # Base directory

In [4]:
inf_data = pd.DataFrame()
gnd_inf_data = pd.DataFrame()

inf_list = []
gnd_inf_list = []

for file in inf_files:

    temp_data = pd.read_json(file)

    inf_list.append(temp_data)

inf_data = pd.concat(inf_list)
inf_data.rename(columns={'obstacle_name': 'closest_obst_name'}, inplace=True)
inf_data.reset_index()

for file in gnd_inf_files:

    temp_data = pd.read_json(file)

    gnd_inf_list.append(temp_data)

gnd_inf_data = pd.concat(gnd_inf_list)
gnd_inf_data.reset_index()

inf_data.rename(columns={'timestamp_cpa': 'time'}, inplace=True)

gnd_inf_data.rename(columns={'timestamp_gprox': 'time'}, inplace=True)
gnd_inf_data = gnd_inf_data.drop(columns=['obstacle_name'])
gnd_inf_data['closest_obst_name'] = 'ground'


In [5]:
gnd_inf_data

,time,icao24,callsign,dip_max,n,ref,url,hour,closest_obst_name
0,2022-08-07 14:11:13,3d0a8c,DECGA,8.0,10.0,3d0a8c_7_070822,https://globe.adsbexchange.com/?icao=3d0a8c&la...,1.659881e+09,ground
1,2022-08-08 11:20:28,3dd3bf,DHALB,8.0,3.0,3dd3bf_1_080822,https://globe.adsbexchange.com/?icao=3dd3bf&la...,1.659956e+09,ground
2,2022-08-11 08:09:14,3cd069,GFD93,92.0,11.0,3cd069_1_110822,https://globe.adsbexchange.com/?icao=3cd069&la...,1.660205e+09,ground
3,2022-08-12 17:30:18,3ff447,DMRFX,8.0,25.0,3ff447_1_120822,https://globe.adsbexchange.com/?icao=3ff447&la...,1.660324e+09,ground
4,2022-08-14 08:59:20,3ff981,DMIPO,24.0,48.0,3ff981_4_140822,https://globe.adsbexchange.com/?icao=3ff981&la...,1.660464e+09,ground
...,...,...,...,...,...,...,...,...,...
10,2022-11-11 08:06:46,3e0f91,DHXCB,237.0,26.0,3e0f91_1_111122,https://globe.adsbexchange.com/?icao=3e0f91&la...,1.668154e+09,ground
11,2022-11-11 10:00:44,4078fb,GNHVI,222.0,23.0,4078fb_1_111122,https://globe.adsbexchange.com/?icao=4078fb&la...,1.668161e+09,ground
12,2022-11-11 10:14:42,3de045,DHFEJ,229.0,32.0,3de045_2_111122,https://globe.adsbexchange.com/?icao=3de045&la...,1.668161e+09,ground
15,2022-11-23 12:12:02,3de8e9,DHILL,54.0,86.0,3de8e9_1_231122,https://globe.adsbexchange.com/?icao=3de8e9&la...,1.669205e+09,ground


In [6]:
inf_data = inf_data.drop(columns=['hour'])
gnd_inf_data = gnd_inf_data.drop(columns=['hour'])

In [13]:
inf_data['positionnac'] = 10
inf_data['version'] = 2

gnd_inf_data['positionnac'] = 10
gnd_inf_data['version'] = 2

In [14]:
ed25519_key = paramiko.Ed25519Key(filename='/home/dimitri/.ssh/id_ed25519')

# Set up the SSH tunnel with the RSA key
with SSHTunnelForwarder(
    (MYSQL_creds['SSH_ADDRESS'], 22),
    ssh_username=MYSQL_creds['SSH_USERNAME'],
    ssh_pkey=ed25519_key,  # Use the loaded RSA key
    remote_bind_address=(MYSQL_creds['REMOTE_BIND_ADDRESS'], MYSQL_creds['REMOTE_BIND_PORT']),
    allow_agent=False,
) as tunnel:
    
    print('Connected')

    engstr =  'mysql+pymysql://' + MYSQL_creds['SSH_USERNAME'] + ':' + MYSQL_creds['PYANYWHERE_PASSWORD'] + '@127.0.0.1:' + str(tunnel.local_bind_port)+'/dme3$' + MYSQL_creds['PROD_DATABASE_NAME']

    engine = create_engine(engstr)

    print('Step 1')

    inf_data.to_sql(con=engine, name = MYSQL_creds['INF_PROD_TABLE_NAME'], if_exists='append')

    print('Step 2')

    gnd_inf_data.to_sql(con=engine, name = MYSQL_creds['GNDINF_PROD_TABLE_NAME'], if_exists='append')

    print("Done")

Connected
Step 1
Step 2
Done
